In [1]:
import sqlite3
import pandas as pd

# Load in Tool Tips 

In [2]:
# load data
df = pd.read_csv("final_airport_info.csv")

# strip whitespace from headerstoll
df.columns = df.columns.str.strip()


con = sqlite3.connect("/Users/devo/sqlite/airport.sqlite")

# drop data into database
df.to_sql("tool_tips",con,if_exists='replace',index = True,index_label=None,dtype = None)

c = con.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE tool_tips RENAME TO new_tool_tips;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    
    CREATE TABLE info (id INTERGER PRIMARY KEY NOT NULL,
                            type TEXT,name TEXT,latitude REAL,
                            longitude REAL,elevation REAL,municipality TEXT,
                            iata_code TEXT,home_link TEXT);

    INSERT INTO info SELECT * FROM new_tool_tips;

    DROP TABLE new_tool_tips;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

c.close()
con.close()

OperationalError: unable to open database file

# Load in Airport Route

In [29]:
df2 = pd.read_csv("clean_data/airport_long_lat.csv")

In [31]:
#get list of uniqure airport 
airport_list= df2['Origin'].unique()
final_df = pd.DataFrame()

In [32]:
for airport in airport_list:
    route = df2[df2['Origin']== airport]
    final_df = final_df.append(route)
    final_df = final_df.drop_duplicates()

In [34]:
final_df.to_csv("clean_data/routes.csv",index=False)

In [3]:
df2 = pd.read_csv("clean_data/airport_long_lat.csv")
df2.columns = df2.columns.str.strip()
con = sqlite3.connect("/Users/devo/sqlite/airport.sqlite")
df2.to_sql("airport_connection",con,if_exists='replace',index = True,index_label=None,dtype = None)
c = con.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;
    BEGIN TRANSACTION;
    
    ALTER TABLE airport_connection RENAME TO new_airport_connection;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    
    CREATE TABLE Airport_Route (id INTERGER PRIMARY KEY NOT NULL,
                            Origin TEXT, Dest TEXT, 
                            Origin_lat REAL,Origin_long REAL,
                            Dest_lat REAL,Dest_long REAL);


    INSERT INTO Airport_Route SELECT * FROM new_airport_connection;

    DROP TABLE new_airport_connection;
    
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

c.close()
con.close()

# Monthly

In [4]:
df3 = pd.read_csv("clean_data/average_monthly_delay.csv")

df3.columns = df3.columns.str.strip()
con = sqlite3.connect("/Users/devo/sqlite/airport_db.sqlite")

df3.to_sql("average",con,if_exists='replace',index = True,index_label=None,dtype = None)
c = con.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;
    BEGIN TRANSACTION;
    
    ALTER TABLE average RENAME TO new_average;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    
    CREATE TABLE Monthly_Average (id INTERGER PRIMARY KEY NOT NULL,
                                    Origin TEXT, Year INTERGER,
                                    Month INTERGER, Depart_Delay REAL,
                                    Arrival_Delay REAL, Carrier_Delay REAL,
                                    Weather_Delay REAL, NAS_Delay REAL, 
                                    Security_Delay REAL, Aircraft_Delay REAL);


    INSERT INTO Monthly_Average SELECT * FROM new_average;

    DROP TABLE new_average;
    
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

c.close()
con.close()

# Delay by Airline

In [5]:
df4 = pd.read_csv("clean_data/Delay_By_Airline.csv")

df4.columns = df4.columns.str.strip()
con = sqlite3.connect("/Users/devo/sqlite/airport_db.sqlite")

df4.to_sql("delay_air",con,if_exists='replace',index = True,index_label=None,dtype = None)
c = con.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;
    BEGIN TRANSACTION;
    
    ALTER TABLE delay_air RENAME TO new_delay_air;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    
    CREATE TABLE Delay_By_Airline (id INTERGER PRIMARY KEY NOT NULL,
                                    Reporting_Airline TEXT, 
                                    Dep_On_Late_Arr REAL, Carrier_Delay REAL,
                                    Weather_Delay REAL, NAS_Delay REAL, 
                                    Security_Delay REAL, Aircraft_Delay REAL, Year INTERGER);


    INSERT INTO Delay_By_Airline SELECT * FROM new_delay_air;

    DROP TABLE new_delay_air;
    
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

c.close()
con.close()